In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os


# 데이터를 가져온 시간을 적어주는 부분
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" }
category_list = ['정치', '경제', '사회', '생활/문화', '세계', 'IT/과학']
#query = input('검색 키워드를 입력하세요 : ') 
news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))
#query = query.replace(' ', '+')


#news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'
news_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100'

req = requests.get(news_url, headers=headers)
soup = BeautifulSoup(req.text, 'html.parser')


news_dict = {}
idx = 0
cur_page = 1

print()
print('크롤링 중...')

while idx < news_num:
### 네이버 뉴스 웹페이지 구성이 바뀌어 태그명, class 속성 값 등을 수정함(20210126) ###
    try : 
        table = soup.find('ul',{'class' : 'list_news'})
        li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
        area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
        a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]

        for n in a_list[:min(len(a_list), news_num-idx)]:
            news_dict[idx] = {'title' : n.get('title'),
                              'url' : n.get('href') }
            idx += 1

        cur_page += 1

        pages = soup.find('div', {'class' : 'sc_page_inner'})
        next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')

        req = requests.get('https://search.naver.com/search.naver' + next_page_url)
        soup = BeautifulSoup(req.text, 'html.parser')
    except Exception as e: 
        continue

print('크롤링 완료')

print('데이터프레임 변환')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_{}_{}.xlsx'.format(query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}'.format(folder_path, xlsx_file_name))
os.startfile(folder_path)

In [1]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os

def get_request(section) : 
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" }
    
    sections = {
        '정치' : 100,
        '경제' : 101,
        '사회' : 102,
        '생활/문화' : 103,
        '세계' : 104,
        'IT/과학' : 105
    }
    
    url = 'https://news.naver.com/main/main/list.nhn'
    
    req = requests.get(url, headers=headers, params={"sid1":sections[section]})
    
    return req

def get_href(soup) : 
    result = []
    
    div = soup.find("div", class_="list_body newsflash_body")
    
    for dt in div.find_all("dt", class_="photo") : 
        result.append(dt.find("a")["href"])
        
    return result 



# 데이터를 가져온 시간을 적어주는 부분
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))

section = input("하나를 입력하세요. : ")

req = get_request(section)
soup = BeautifulSoup(req.text, 'html.parser')

list_href = get_href(soup)

총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : 100
하나를 입력하세요. : 정치


AttributeError: 'NoneType' object has no attribute 'find_all'